- 数据概览：
    - 训练集共891个样本，其中549个未存活、342个存活；
    - 测试集共418个样本。

|序号|表头名称|表头含义|取值类型|
|--|--|--|--|
|1|PassengerID|旅客id|整数|
|2|Survived|是否存活|0：否； 1：是|
|3|Pclass|舱位|1：头等舱； 2：二等舱； 3：三等舱|
|4|Name|旅客姓名|字符串|
|5|Sex|性别|female；male|
|6|Age|年龄|年龄小于1则为分数；如果是估计年龄，则为xx.5；有缺失值|
|7|SibSp|乘客在船上兄弟姐妹/配偶的个数|整数|
|8|Parch|乘客在船上父母/孩子的个数|整数|
|9|Ticket|票号|字符串|
|10|Fare|票价|浮点数|
|11|Cabin|船舱|字符串；有缺失值|
|12|Embarked|登船港口|C： Cherbourg；Q：Queenstown；S：Southampton；有缺失值|

In [ ]:
import numpy as np
import pandas as pd
from pylab import mpl

# 指定默认字体：解决plot不能显示中文问题
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus'] = False               # 解决保存图像是负号'-'显示为方块的问题

# 加载数据
data_train = pd.read_csv('./data/train.csv')
data_test = pd.read_csv('./data/test.csv')


In [ ]:
print(data_train.info())
print(data_test.info())


- 属性概览
  - 数值型数据：Age(有缺失值：训练集177个样本缺失，测试集86个样本缺失)、SibSp、Parch、Fare(测试集1个样本缺失)；
   - 类别型数据：Pclass、Sex、Embarked(有缺失值：训练集2个样本缺失，测试集无缺失)；
   - ~~文本型数据：Name、Ticket、Cabin(有缺失值：训练集687个样本缺失，327个样本缺失)。~~
- 删除Name、Ticket、Cabin，暂定保留Age、SibSp、Parch、Fare、Pclass、Sex、Embarked作为属性值
- 训练集中只有两个样本的Embarked值缺失，删除这两个样本
- Pclass、Sex、Embarked进行one-hot编码
- 训练集中缺失Age值的样本过多，使用knn插值

In [ ]:
# 删除PassengerId、Name、Ticket、Cabin
data_train = data_train.drop(
    columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])


In [ ]:
# 删除Embarked值缺失的样本
nan_index = np.where(pd.isna(data_train['Embarked']))
data_train = data_train.drop(index=np.array(nan_index).reshape(-1))


In [ ]:
# Pclass、Sex、Embarked进行one-hot编码
pclass = pd.get_dummies(data_train['Pclass'], prefix='Pclass')
sex = pd.get_dummies(data_train['Sex'], prefix='Sex')
embarked = pd.get_dummies(data_train['Embarked'], prefix='Embarked')

y_train = data_train['Survived']
x_train = data_train.drop(columns=['Survived'])
x_train = x_train.drop(columns=['Pclass', 'Sex', 'Embarked'])
x_train = pd.concat([x_train, pclass, sex, embarked], axis=1)


In [ ]:
# 对Age属性使用knn插值
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=10)
x_train = knn_imputer.fit_transform(x_train)


- 选择模型
- 训练模型（十折交叉验证）